# Conway Examples

The purpose of this notebook is to experiment with different algorithms for
calculating a generation of a Conway's Game of Life grid.  All examples will
include a function, an input, and a call to the function with the input to
produce an output.

These examples are not test-driven but are designed only to show common
algorithmic approaches.

## Discussion

The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells,
each of which is in one of two possible states, alive or dead, or "populated" or "unpopulated".
Every cell interacts with its eight neighbours, which are the cells that are horizontally,
vertically, or diagonally adjacent. At each step in time, the following transitions occur:

* DIES by UNDERPOPULATION:  Any live cell with fewer than two live neighbours dies, as if caused by underpopulation.
* LIVES:                    Any live cell with two or three live neighbours lives on to the next generation.
* DIES by OVERPOPULATION:   Any live cell with more than three live neighbours dies, as if by overpopulation.
* REPRODUCES:               Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

The initial pattern constitutes the seed of the system. The first generation is created by
applying the above rules simultaneously to every cell in the seed-births and deaths occur
simultaneously, and the discrete moment at which this happens is sometimes called a tick
(in other words, each generation is a pure function of the preceding one). The rules
continue to be applied repeatedly to create further generations.

### Object Oriented Two-Dimensional Array

In [1]:
class GridEvaluator(object):

    def numberOfLivingNeighbors(self, grid, x, y):
        livingNeighbors = 0

        startx = x if (x == 0) else x - 1
        endx = x if (x == len(grid[0]) - 1) else x + 1
        starty = y if (y == 0) else y - 1
        endy = y if (y == len(grid) - 1) else y + 1

        for rowIndex in range(starty,endy+1):
            for columnIndex in range(startx, endx+1):
                if (grid[rowIndex][columnIndex] == True and (rowIndex != y or columnIndex != x)):
                    livingNeighbors = livingNeighbors + 1

        return livingNeighbors

In [9]:
GRID1 = [
    [True, False],
    [False, False]
]

GRID2 = [
    [True, True],
    [False, False]
]

GRID3 = [
    [True, True],
    [True, False]
]

GRID4 = [
    [True, True, True],
    [True, False, True],
    [True, True, True]
]

def test_numberOfLivingNeighbors_returns_zero_when_no_neighbors(evaluator):
    assert 0 == evaluator.numberOfLivingNeighbors([[True]], 0, 0)

def test_numberOfLivingNeighbors_returns_zero_when_no_neighbors_are_alive(evaluator):
    assert 0 == evaluator.numberOfLivingNeighbors(GRID1, 0, 0)

def test_numberOfLivingNeighbors_returns_one_when_one_neighbor_is_alive(evaluator):
    assert 1 == evaluator.numberOfLivingNeighbors(GRID1, 0, 1)

def test_numberOfLivingNeighbors_returns_two_when_two_neighbors_are_alive(evaluator):
    assert 2 == evaluator.numberOfLivingNeighbors(GRID2, 0, 1)

def test_numberOfLivingNeighbors_returns_three_when_three_neighbors_are_alive(evaluator):
    assert 3 == evaluator.numberOfLivingNeighbors(GRID3, 1, 1)

def test_numberOfLivingNeighbors_returns_eight_when_eight_neighbors_are_alive(evaluator):
    assert 8 == evaluator.numberOfLivingNeighbors(GRID4, 1, 1)
    
gridEvaluator = GridEvaluator()

test_numberOfLivingNeighbors_returns_zero_when_no_neighbors(gridEvaluator)
test_numberOfLivingNeighbors_returns_zero_when_no_neighbors_are_alive(gridEvaluator)
test_numberOfLivingNeighbors_returns_one_when_one_neighbor_is_alive(gridEvaluator)
test_numberOfLivingNeighbors_returns_two_when_two_neighbors_are_alive(gridEvaluator)
test_numberOfLivingNeighbors_returns_three_when_three_neighbors_are_alive(gridEvaluator)
test_numberOfLivingNeighbors_returns_eight_when_eight_neighbors_are_alive(gridEvaluator)

In [10]:
from copy import deepcopy

class GridCalculator(object):

    gridEvaluator = GridEvaluator()

    def update(self, current):
        # currentPlayingField.eachWithIndex { List<Boolean> row, int y ->
        #     row.eachWithIndex { Boolean cell, int x ->

        updated = deepcopy(current)

        for y, row in enumerate(current):
            for x, cell in enumerate(row):
                if (cell is True) and (self.gridEvaluator.numberOfLivingNeighbors(current, x, y) not in [2,3]):
                    updated[y][x] = False
                elif (cell is False) and (self.gridEvaluator.numberOfLivingNeighbors(current, x, y) == 3):
                    updated[y][x] = True
                else:
                    pass
        return updated

In [27]:
DIES_BY_UNDERPOPULATION_BEFORE = [
    [True, True],
    [False, False]
]

DIES_BY_UNDERPOPULATION_AFTER = [
    [False, False],
    [False, False]
]

LIVES_BY_NOT_UNDERPOPULATED = [
    [True, True],
    [True, False]
]

LIVES_BY_NOT_UNDERPOPULATED2 = [
    [True, True],
    [True, True]
]

DIES_WHEN_OVERPOPULATED_BEFORE = [
    [False, True, False],
    [True, True, True],
    [False, True, False]

]

DIES_WHEN_OVERPOPULATED_AFTER = [
    [True, True, True], 
    [True, False, True], 
    [True, True, True]
]

def test_update_unchanged_if_empty(calculator):
    current = [[]]
    assert current == calculator.update(current)

def test_update_dies_when_underpopulated_simplest(calculator):
    current = [[True]]
    assert [[False]] == calculator.update(current)

def test_update_dies_when_underpopulatied(calculator):
    assert DIES_BY_UNDERPOPULATION_AFTER == calculator.update(DIES_BY_UNDERPOPULATION_BEFORE)

def test_update_lives_when_two_neighbors(calculator):
    assert [[False, True, False]] == calculator.update([[True, True, True]])

def test_update_lives_when_three_neighbors(calculator):
    assert LIVES_BY_NOT_UNDERPOPULATED2 == calculator.update(LIVES_BY_NOT_UNDERPOPULATED2)

def test_update_dies_when_overpopulated(calculator):
    assert DIES_WHEN_OVERPOPULATED_AFTER == calculator.update(DIES_WHEN_OVERPOPULATED_BEFORE)

def test_update_reproduces(calculator):
    assert LIVES_BY_NOT_UNDERPOPULATED2 == calculator.update(LIVES_BY_NOT_UNDERPOPULATED)
    
gridCalculator = GridCalculator()

test_update_dies_when_underpopulated_simplest(gridCalculator)
test_update_dies_when_underpopulatied(gridCalculator)
test_update_lives_when_two_neighbors(gridCalculator)
test_update_lives_when_three_neighbors(gridCalculator)
test_update_dies_when_overpopulated(gridCalculator)
test_update_reproduces(gridCalculator)
